In [ ]:
dbutils.fs.ls('dbfs:/databricks-datasets/nyctaxi/tripdata/green')

Out[158]: [FileInfo(path='dbfs:/databricks-datasets/nyctaxi/tripdata/green/green_tripdata_2013-08.csv.gz', name='green_tripdata_2013-08.csv.gz', size=303960, modificationTime=1590475131000),
 FileInfo(path='dbfs:/databricks-datasets/nyctaxi/tripdata/green/green_tripdata_2013-09.csv.gz', name='green_tripdata_2013-09.csv.gz', size=2022534, modificationTime=1590475133000),
 FileInfo(path='dbfs:/databricks-datasets/nyctaxi/tripdata/green/green_tripdata_2013-10.csv.gz', name='green_tripdata_2013-10.csv.gz', size=6852107, modificationTime=1590475136000),
 FileInfo(path='dbfs:/databricks-datasets/nyctaxi/tripdata/green/green_tripdata_2013-11.csv.gz', name='green_tripdata_2013-11.csv.gz', size=15387328, modificationTime=1590475142000),
 FileInfo(path='dbfs:/databricks-datasets/nyctaxi/tripdata/green/green_tripdata_2013-12.csv.gz', name='green_tripdata_2013-12.csv.gz', size=24075318, modificationTime=1590475145000),
 FileInfo(path='dbfs:/databricks-datasets/nyctaxi/tripdata/green/green_tripdata

In [ ]:
BRONZE

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2059692673858779>:1
----> 1 BRONZE

NameError: name 'BRONZE' is not defined

In [ ]:
import re
import pandas as pd
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("LoadNYCTaxiData").getOrCreate()

# List of file paths
file_paths = [
    'dbfs:/databricks-datasets/nyctaxi/tripdata/green/green_tripdata_2019-08.csv.gz',
    'dbfs:/databricks-datasets/nyctaxi/tripdata/green/green_tripdata_2019-09.csv.gz',
    'dbfs:/databricks-datasets/nyctaxi/tripdata/green/green_tripdata_2019-10.csv.gz',
    'dbfs:/databricks-datasets/nyctaxi/tripdata/green/green_tripdata_2019-11.csv.gz',
    'dbfs:/databricks-datasets/nyctaxi/tripdata/green/green_tripdata_2019-12.csv.gz'
]

# Load data using read.format().load() for each file
dataframes = []
for file_path in file_paths:
    df = spark.read.format("csv").option("header", "true").load(file_path)
    dataframes.append(df)

# Perform operations on DataFrames
for df in dataframes:
    df.show()




+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2019-08-01 00:22:12|  2019-08-01 00:22:38|                 N|         1|         145|         145|              1|          .00|        2.5|    0|    0.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
import re

cleaned_dataframes = []

def clean_and_lower_string(input_string):
    # Remove invalid characters and convert to lowercase
    cleaned_string = re.sub(r'[ ,;{}()\n\t=]', '', str(input_string)).lower()
    return cleaned_string

# Assuming you have a list of DataFrames named dataframes
for i, df in enumerate(dataframes):
    column_names = df.columns

    # Create a UDF for the clean_and_lower_string function
    clean_and_lower_udf = udf(clean_and_lower_string, StringType())

    # Apply the UDF to each column name and create a new DataFrame with cleaned column names
    cleaned_df = df.select([clean_and_lower_udf(col(column)).alias(clean_and_lower_string(column)) for column in column_names])

    cleaned_dataframes.append(cleaned_df)

    # Show the cleaned DataFrame
    print(f"Cleaned DataFrame {i+1}:")
    cleaned_df.show()



Cleaned DataFrame 1:
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|vendorid|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|ratecodeid|pulocationid|dolocationid|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2|  2019-08-0100:22:12|   2019-08-0100:22:38|                 n|         1|         145|         145|              1|          .00|   

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
import re


# Merge DataFrames using union
merged_cleaned_df = None
for df in cleaned_dataframes:
    if merged_cleaned_df is None:
        merged_cleaned_df = df
    else:
        merged_cleaned_df = merged_cleaned_df.union(df)

# Show the merged cleaned DataFrame
merged_cleaned_df.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|vendorid|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|ratecodeid|pulocationid|dolocationid|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2|  2019-08-0100:22:12|   2019-08-0100:22:38|                 n|         1|         145|         145|              1|          .00|        2.5|    0|    0.

In [ ]:
# Assuming you have a DataFrame named merged_cleaned_df
merged_cleaned_df.createOrReplaceTempView("my_temp_view")

# Now you can use SQL queries on the temporary view
spark.sql("SELECT * FROM my_temp_view").show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|vendorid|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|ratecodeid|pulocationid|dolocationid|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2|  2019-08-0100:22:12|   2019-08-0100:22:38|                 n|         1|         145|         145|              1|          .00|        2.5|    0|    0.

In [ ]:
from delta import DeltaTable

spark.sql("SELECT * FROM my_temp_view")

# Create a Delta table using CTAS
spark.sql('''
    CREATE TABLE bronze_green_trip_data4
    USING delta
    AS
    SELECT *
    FROM my_temp_view
''')

spark.sql('''SELECT *
    FROM my_temp_view''')



Out[6]: DataFrame[vendorid: string, lpep_pickup_datetime: string, lpep_dropoff_datetime: string, store_and_fwd_flag: string, ratecodeid: string, pulocationid: string, dolocationid: string, passenger_count: string, trip_distance: string, fare_amount: string, extra: string, mta_tax: string, tip_amount: string, tolls_amount: string, ehail_fee: string, improvement_surcharge: string, total_amount: string, payment_type: string, trip_type: string, congestion_surcharge: string]

In [ ]:
result = spark.sql("SELECT * FROM bronze_green_trip_data4 WHERE columntripdistance is null")
result.show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2059692673858777>:1
----> 1 result = spark.sql("SELECT * FROM bronze_green_trip_data4 WHERE columntripdistance is null")
      2 result.show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/session.py:1387, in SparkSession.sql(self, sqlQuery, args, **kwargs)
   1385 try:
   1386     litArgs = {k: _to_java_column(lit(v)) for k, v in (args or {}).items()}
-> 1387     return DataFrame(self._jsparkSession.sql(sqlQuery, litArgs), self)
   1388 finally:
   1389     i

In [ ]:
SILVER

In [ ]:
table_name = "bronze_green_trip_data4"
delta_table = spark.table(table_name)

# Show the contents of the Delta table
delta_table.show(100)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|vendorid|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|ratecodeid|pulocationid|dolocationid|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2|  2019-08-0100:22:12|   2019-08-0100:22:38|                 n|         1|         145|         145|              1|          .00|        2.5|    0|    0.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum, when

column_names = delta_table.columns

row_count = delta_table.count()
 
for column in column_names:
    print(f"Processing column: {column}")
    
    # Calculate the total number of values and the number of null values in the column
    none_count = delta_table.select(sum(when(col(column) == 'none', 1).otherwise(0))).first()[0]
    procentas = (none_count / row_count)*100
    if procentas >40:
        columtodelete = column



    print(f"Total values in {column}: {row_count}")
    print(f"'none' values in {column}: {none_count}")
    print(f"'none percentage in {column}: {procentas}")


    print("")
print(columtodelete)

columns_to_keep = [col for col in delta_table.columns if col != columtodelete]
new_delta_table = delta_table.select(columns_to_keep)

new_delta_table.show()



Processing column: vendorid
Total values in vendorid: 2275271
'none' values in vendorid: 376740
'none percentage in vendorid: 16.55802759319659

Processing column: lpep_pickup_datetime
Total values in lpep_pickup_datetime: 2275271
'none' values in lpep_pickup_datetime: 0
'none percentage in lpep_pickup_datetime: 0.0

Processing column: lpep_dropoff_datetime
Total values in lpep_dropoff_datetime: 2275271
'none' values in lpep_dropoff_datetime: 0
'none percentage in lpep_dropoff_datetime: 0.0

Processing column: store_and_fwd_flag
Total values in store_and_fwd_flag: 2275271
'none' values in store_and_fwd_flag: 376740
'none percentage in store_and_fwd_flag: 16.55802759319659

Processing column: ratecodeid
Total values in ratecodeid: 2275271
'none' values in ratecodeid: 376740
'none percentage in ratecodeid: 16.55802759319659

Processing column: pulocationid
Total values in pulocationid: 2275271
'none' values in pulocationid: 0
'none percentage in pulocationid: 0.0

Processing column: dolo

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr

distinct_table = new_delta_table.distinct()

distinct_table.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------------------+
|vendorid|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|ratecodeid|pulocationid|dolocationid|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------------------+
|       2|  2019-10-0100:00:48|   2019-10-0100:05:42|                 n|         1|         210|         108|              2|         1.03|          6|  0.5|    0.5|      2.19|           0|    

In [ ]:

distinct_table.createOrReplaceTempView("SilverView")

# Now you can use SQL queries on the temporary view
spark.sql("SELECT * FROM SilverView").show()

# Create a Delta table using CTAS
spark.sql('''
    CREATE TABLE silver_green_trip_data4
    USING delta
    AS
    SELECT *
    FROM SilverView
''')


+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------------------+
|vendorid|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|ratecodeid|pulocationid|dolocationid|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------------------+
|       2|  2019-10-0100:00:48|   2019-10-0100:05:42|                 n|         1|         210|         108|              2|         1.03|          6|  0.5|    0.5|      2.19|           0|    

In [ ]:
table_name_silver = "silver_green_trip_data4"
delta_table_silver = spark.table(table_name_silver)

# Show the contents of the Delta table
delta_table_silver.show(100)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------------------+
|vendorid|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|ratecodeid|pulocationid|dolocationid|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------------------+
|       2|  2019-10-0100:00:48|   2019-10-0100:05:42|                 n|         1|         210|         108|              2|         1.03|          6|  0.5|    0.5|      2.19|           0|    

In [ ]:
GOLD

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col


# Convert column names to snake_case
snake_case_columns = [col(column).alias(column.lower().replace(' ', '_')) for column in delta_table_silver.columns]

# Select the DataFrame with snake_case column names
transformed_df = delta_table_silver.select(snake_case_columns)

# Show the transformed DataFrame
transformed_df.show()


+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------------------+
|vendorid|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|ratecodeid|pulocationid|dolocationid|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------------------+
|       2|  2019-10-0100:00:48|   2019-10-0100:05:42|                 n|         1|         210|         108|              2|         1.03|          6|  0.5|    0.5|      2.19|           0|    

In [ ]:
# Print the original column names
print("Original Column Names:")
for column in delta_table_silver.columns:
    print(column)


Original Column Names:
vendorid
lpep_pickup_datetime
lpep_dropoff_datetime
store_and_fwd_flag
ratecodeid
pulocationid
dolocationid
passenger_count
trip_distance
fare_amount
extra
mta_tax
tip_amount
tolls_amount
improvement_surcharge
total_amount
payment_type
trip_type
congestion_surcharge


In [ ]:
# Rename the columns
renamed_df = delta_table_silver.withColumnRenamed("lpep_pickup_datetime", "pickup_datetime").withColumnRenamed("lpep_dropoff_datetime", "dropoff_datetime")

# Show the DataFrame with the renamed columns
renamed_df.show()


# Print the original column names
print("Original Column Names:")
for column in renamed_df.columns:
    print(column)
renamed_df.show()

+--------+------------------+------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------------------+
|vendorid|   pickup_datetime|  dropoff_datetime|store_and_fwd_flag|ratecodeid|pulocationid|dolocationid|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+------------------+------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------------------+
|       2|2019-10-0100:00:48|2019-10-0100:05:42|                 n|         1|         210|         108|              2|         1.03|          6|  0.5|    0.5|      2.19|           0|                  0.3|  

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_unixtime, unix_timestamp, concat_ws, substring

# Initialize Spark session
spark = SparkSession.builder.appName("DateTimeConversion").getOrCreate()

# Assuming you have a DataFrame named renamed_df with columns like "pickup_datetime" and "dropoff_datetime"
# You can replace this with your actual DataFrame creation or loading process

# Define the columns to iterate through
columns_to_iterate = ["pickup_datetime", "dropoff_datetime"]

# Iterate through each value of the specified columns and convert the datetime format
for column_name in columns_to_iterate:
    transformed_column = (
        from_unixtime(
            unix_timestamp(
                concat_ws(
                    " ",
                    substring(col(column_name), 1, 10),
                    substring(col(column_name), 11, 8)
                ),
                "yyyy-MM-dd HH:mm:ss"
            )
        ).alias(column_name)
    )
    renamed_df = renamed_df.withColumn(column_name, transformed_column)

# Show the DataFrame with the converted datetime format
renamed_df.show()


+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------------------+
|vendorid|    pickup_datetime|   dropoff_datetime|store_and_fwd_flag|ratecodeid|pulocationid|dolocationid|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------------------+
|       2|2019-10-01 00:00:48|2019-10-01 00:05:42|                 n|         1|         210|         108|              2|         1.03|          6|  0.5|    0.5|      2.19|           0|                

In [ ]:
from pyspark.sql.functions import col, unix_timestamp, round

# Calculate the trip duration in minutes and round to 1 decimal place
renamed_df = renamed_df.withColumn(
    "trip_duration_minutes",
    round((unix_timestamp("dropoff_datetime") - unix_timestamp("pickup_datetime")) / 60, 1)
)

# Show the updated DataFrame with the new column
renamed_df.show()


+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------------------+---------------------+
|vendorid|    pickup_datetime|   dropoff_datetime|store_and_fwd_flag|ratecodeid|pulocationid|dolocationid|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|trip_duration_minutes|
+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------------------+---------------------+
|       2|2019-10-01 00:00:48|2019-10-01 00:05:42|                 n|         1|         210|         108|              2|         1.03|

In [ ]:
renamed_df = renamed_df[["vendorid",
"pickup_datetime",
"dropoff_datetime",
"trip_duration_minutes",
"store_and_fwd_flag",
"ratecodeid",
"pulocationid",
"dolocationid",
"passenger_count",
"trip_distance",
"fare_amount",
"extra",
"mta_tax",
"tip_amount",
"tolls_amount",
"improvement_surcharge",
"total_amount",
"payment_type",
"trip_type",
"congestion_surcharge"]]

renamed_df.show()


+--------+-------------------+-------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------------------+
|vendorid|    pickup_datetime|   dropoff_datetime|trip_duration_minutes|store_and_fwd_flag|ratecodeid|pulocationid|dolocationid|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+-------------------+-------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------------------+
|       2|2019-10-01 00:00:48|2019-10-01 00:05:42|                  4.9|                 n|         1|         210|         108|        

In [ ]:
from pyspark.sql.functions import sum, when, col

# Iterate through each column and calculate the count of 'none' values
none_counts = {}
for column_name in renamed_df.columns:
    none_count = renamed_df.select(sum(when(col(column_name) == 'none', 1).otherwise(0))).first()[0]
    none_counts[column_name] = none_count

# Print the counts of 'none' values in each column
for column_name, count in none_counts.items():
    print(f"Column: {column_name}, None Count: {count}")


Column: vendorid, None Count: 376740
Column: pickup_datetime, None Count: 0
Column: dropoff_datetime, None Count: 0
Column: trip_duration_minutes, None Count: 0
Column: store_and_fwd_flag, None Count: 376740
Column: ratecodeid, None Count: 376740
Column: pulocationid, None Count: 0
Column: dolocationid, None Count: 0
Column: passenger_count, None Count: 376740
Column: trip_distance, None Count: 0
Column: fare_amount, None Count: 0
Column: extra, None Count: 0
Column: mta_tax, None Count: 0
Column: tip_amount, None Count: 0
Column: tolls_amount, None Count: 0
Column: improvement_surcharge, None Count: 2
Column: total_amount, None Count: 0
Column: payment_type, None Count: 376740
Column: trip_type, None Count: 376827
Column: congestion_surcharge, None Count: 376742


In [ ]:
from pyspark.sql.functions import col

# Filter out rows with 'none' values in any column
filtered_df = renamed_df
for column_name in renamed_df.columns:
    filtered_df = filtered_df.filter(col(column_name) != 'none')

# Calculate the number of rows before and after filtering
total_rows_before = renamed_df.count()
total_rows_after = filtered_df.count()

print(f"Total rows before filtering 'none' values: {total_rows_before}")
print(f"Total rows after filtering 'none' values: {total_rows_after}")


Total rows before filtering 'none' values: 2275267
Total rows after filtering 'none' values: 0


In [ ]:
from pyspark.sql.functions import col, trim

filtered_df = renamed_df
for column_name in renamed_df.columns:
    filtered_df = filtered_df.filter(trim(col(column_name)) != 'none')

total_rows_before = renamed_df.count()
total_rows_after = filtered_df.count()

print(f"Total rows before filtering 'none' values: {total_rows_before}")
print(f"Total rows after filtering 'none' values: {total_rows_after}")


Total rows before filtering 'none' values: 2275267
Total rows after filtering 'none' values: 1898440


In [ ]:
filtered_df.show()
renamed_df.show()

+--------+-------------------+-------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------------------+
|vendorid|    pickup_datetime|   dropoff_datetime|trip_duration_minutes|store_and_fwd_flag|ratecodeid|pulocationid|dolocationid|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+-------------------+-------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------------------+
|       2|2019-10-01 00:00:48|2019-10-01 00:05:42|                  4.9|                 n|         1|         210|         108|        

In [ ]:
# Assuming you have a DataFrame named merged_cleaned_df
filtered_df.createOrReplaceTempView("my_temp_viewGold")

# Now you can use SQL queries on the temporary view
spark.sql("SELECT * FROM my_temp_viewGold").show()

+--------+-------------------+-------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------------------+
|vendorid|    pickup_datetime|   dropoff_datetime|trip_duration_minutes|store_and_fwd_flag|ratecodeid|pulocationid|dolocationid|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+-------------------+-------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------------------+
|       2|2019-10-01 00:00:48|2019-10-01 00:05:42|                  4.9|                 n|         1|         210|         108|        

In [ ]:
spark.sql("SELECT * FROM my_temp_viewGold")

Out[38]: DataFrame[vendorid: string, pickup_datetime: string, dropoff_datetime: string, trip_duration_minutes: double, store_and_fwd_flag: string, ratecodeid: string, pulocationid: string, dolocationid: string, passenger_count: string, trip_distance: string, fare_amount: string, extra: string, mta_tax: string, tip_amount: string, tolls_amount: string, improvement_surcharge: string, total_amount: string, payment_type: string, trip_type: string, congestion_surcharge: string]

In [ ]:
from delta import DeltaTable

spark.sql("SELECT * FROM my_temp_viewGold")

# Create a Delta table using CTAS
spark.sql('''
    CREATE TABLE gold_green_trip_data
    USING delta
    AS
    SELECT *
    FROM my_temp_viewGold
''')




Out[32]: DataFrame[vendorid: string, pickup_datetime: string, dropoff_datetime: string, trip_duration_minutes: double, store_and_fwd_flag: string, ratecodeid: string, pulocationid: string, dolocationid: string, passenger_count: string, trip_distance: string, fare_amount: string, extra: string, mta_tax: string, tip_amount: string, tolls_amount: string, improvement_surcharge: string, total_amount: string, payment_type: string, trip_type: string, congestion_surcharge: string]

In [ ]:
spark.sql('''SELECT *
    FROM gold_green_trip_data''')

Out[34]: DataFrame[vendorid: string, pickup_datetime: string, dropoff_datetime: string, trip_duration_minutes: double, store_and_fwd_flag: string, ratecodeid: string, pulocationid: string, dolocationid: string, passenger_count: string, trip_distance: string, fare_amount: string, extra: string, mta_tax: string, tip_amount: string, tolls_amount: string, improvement_surcharge: string, total_amount: string, payment_type: string, trip_type: string, congestion_surcharge: string]

In [ ]:
table_name_gold = "gold_green_trip_data"
delta_table_gold = spark.table(table_name_gold)

# Show the contents of the Delta table
delta_table_gold.show(100)

+--------+-------------------+-------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------------------+
|vendorid|    pickup_datetime|   dropoff_datetime|trip_duration_minutes|store_and_fwd_flag|ratecodeid|pulocationid|dolocationid|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+-------------------+-------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+---------+--------------------+
|       2|2019-10-01 00:00:48|2019-10-01 00:05:42|                  4.9|                 n|         1|         210|         108|        